# Histogram Utilities
> Summary description here

In [ ]:
# default_exp histogram_utils

In [ ]:
#hide
from cellsegment.core import *
from nbdev.showdoc import *


In [ ]:
# export
import random
from cellsegment.core import *
from  cellsegment.dataprep import *
import json
import shutil

import pandas
from fastai.vision import get_image_files, parallel, partial, PIL, Path, get_files
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb
from skimage.exposure import histogram
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os

In [ ]:
# export
def show_histogram(im, figsize=None, ax=None, color=None, rgb=False, scale=None, label=None, title=None):
    if not ax: fig, ax = plt.subplots(figsize=figsize)
    if scale:
        hist, hist_centers = histogram(im*scale[1])
        hist = (hist*(scale[0]/hist.max())).astype(int)
    else:
        hist, hist_centers = histogram(im)

    ax.plot(hist_centers, hist, color=color, label=label)
    if label: ax.legend()
    if title: plt.title(title)
    return ax

In [ ]:
# export
def plot_color_histogram(img):
    colors = ("b", "g", "r")
    plt.figure()
    plt.title("'Flattened' Color Histogram")
    plt.xlabel("Bins")
    plt.ylabel("# of Pixels")
    features = []
    chans = [img[:,:,i]*255 for i in range(3)]
    for (chan, color) in zip(chans, colors):
        hist, hist_centers = histogram(img)
        plt.plot(hist_centers, hist, color = color)
        

In [ ]:
# export
def plot_well_histograms(img, img_gray, img_bw, well_regionprops):
    well_circle_mask = pad(well_regionprops.convex_image, (img.shape[:2]), well_regionprops.bbox[:2])
    im = img.copy()
    im[~img_bw]=0
    hist0, hist_centers = histogram(im)
    im = img.copy()
    im[~well_circle_mask]=0
    hist1, hist_centers = histogram(im)
    im = img.copy()
    im[img_gray<0.3]  = 0
    hist2, hist_centers = histogram(im)
    hist0[0] = 0
    hist1[0] = 0
    hist2[0] = 0
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.plot(hist_centers, hist0, label='thresholded well pixels only')
    ax.plot(hist_centers, hist1, label='all well pixels')
    ax.plot(hist_centers, hist2, label='entire image')
    ax.set_title('histogram of gray values')
    ax.legend()
        